In [72]:
# https://medium.com/analytics-vidhya/feature-selection-using-scikit-learn-5b4362e0c19b
# https://towardsdatascience.com/rank-the-features-now-rank-again-4dafd8cde3c8

In [2]:
ROOT_DIR = ".."
DATASET_DIR = "{}/datasets".format(ROOT_DIR)
DATASET_DIR

'../datasets'

In [3]:
### Use LightGBM

# ### Using ML/DL libraries
# 1. OpenChem
# 2. ChemProp
# 3. DeepChem

In [4]:
import os
import sys
ROOT_DIR = os.pardir
sys.path.insert(0, os.path.abspath(ROOT_DIR))
from matplotlib import pyplot
import numpy as np
import pandas as pd
from pprint import pprint
import re

from scipy import stats
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV


In [5]:
def detect_outlier_z_scores(df):
  """
  To perform outlier detection, we are going to employ the Z-Score method because it is the simplest one.
  This s a slight modification of the code from the following link
  https://www.kaggle.com/alexandrehsd/binary-multiclass-classification-factor-analysis/notebookSS
  """
  flag_outlier = False

  for feature in df:
    #print(feature)
    column = df[feature]
    mean = np.mean(column)
    std = np.std(column)
    z_scores = (column - mean) / std
    outliers = np.abs(z_scores) > 3
    
    n_outliers = sum(outliers)
    
    if n_outliers > 0:
      print("{} has {} outliers".format(feature, n_outliers))
      flag_outlier = True

  if not flag_outlier:
    print("\nThe dataset has no outliers.")
    
    return None

def remove_outliers_by_z_score(df:pd.DataFrame, threshold:int = 3):
    ## Find outliers for all features
    z = np.abs(stats.zscore(df))
    outliers = np.where(z > threshold)
    columns = df.columns.tolist()
    cols_with_outliers = [columns[i] for i in 
                         set(outliers[1].tolist())]
    
    print("Features with outliers ({}) : {}".format(len(cols_with_outliers), cols_with_outliers))
    print(outliers[0].size)
    
    ## Remove outliers
    print("\nRemoving {} rows...".format(  len(set(outliers[0].tolist()))   ))
    print(np.where(z <= threshold)[0].size)
    new_df = df[(z <= threshold).all(axis=1)]
    print(new_df.shape)
    return new_df

In [6]:
dataset = pd.read_csv("{}/csv/nr-ahr.csv".format(DATASET_DIR))
features = dataset.columns.tolist()
target = "Activity"
dataset.head()

,ALogP (#1),ALogP (#2),ALogP (#3),AMW,Aromatic Atoms Count,Aromatic Bonds Count,Atomic Polarizabilities,BCUT (#1),BCUT (#10),BCUT (#100),...,smr_VSA10,smr_VSA2,smr_VSA3,smr_VSA4,smr_VSA5,smr_VSA6,smr_VSA7,smr_VSA8,smr_VSA9,Activity
0,1.6575,2.747306,107.0444,396.571,6,6,72.170548,15.996934,11.999,0.0,...,5.969305,0.0,0.000000,23.168709,95.995469,0.000000,29.326004,0.0,5.749512,0
1,2.8230,7.969329,48.0370,166.220,6,6,28.539102,15.998261,11.999,0.0,...,0.000000,0.0,0.000000,0.000000,26.186202,0.000000,23.762553,0.0,11.499024,1
2,2.8230,7.969329,48.0370,166.220,6,6,28.539102,15.998261,11.999,0.0,...,0.000000,0.0,0.000000,0.000000,26.186202,0.000000,23.762553,0.0,11.499024,1
3,2.2844,5.218483,108.6492,329.467,12,12,59.349825,14.009292,11.999,0.0,...,16.972176,0.0,0.000000,0.000000,0.000000,33.090598,95.601392,0.0,0.000000,1
4,1.1843,1.402566,95.3407,354.450,9,10,58.902618,15.996934,11.999,0.0,...,16.872230,0.0,9.883888,17.753718,37.829094,20.199310,35.522848,0.0,0.000000,1


In [7]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1900 entries, 0 to 1899
Columns: 1608 entries, ALogP (#1) to Activity
dtypes: float64(483), int64(1125)
memory usage: 23.3 MB


In [8]:
dataset.isna().any()

ALogP (#1)               True
ALogP (#2)               True
ALogP (#3)               True
AMW                     False
Aromatic Atoms Count    False
                        ...  
smr_VSA6                False
smr_VSA7                False
smr_VSA8                False
smr_VSA9                False
Activity                False
Length: 1608, dtype: bool

In [9]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
import re


print(dataset.shape)
dataset.dropna(inplace=True)
print(dataset.shape)

X = dataset[dataset.columns.difference([target])]
y = dataset[target]
pattern = re.compile("BCUT|PubChem")
cols_bcut_pubchem = [x for x in X.columns.tolist() if not pattern.match(x) is None]
#print(len(cols_bcut_pubchem))
#X_mini = X[X.columns.difference(cols_bcut_pubchem)]
#print(X_mini.shape)


ahr_corr = X.corr()

(1900, 1608)
(1882, 1608)


In [10]:
ahr_corr_pairs = ahr_corr.where(ahr_corr>=0.8).stack().index.values
ahr_corr_pairs[:10]
# ahr_corr_high.where(ahr_corr_high>=0.8).stack().index.values

array([('ALogP (#1)', 'ALogP (#1)'), ('ALogP (#2)', 'ALogP (#2)'),
       ('ALogP (#3)', 'ALogP (#3)'), ('ALogP (#3)', 'AMW'),
       ('ALogP (#3)', 'Atomic Polarizabilities'),
       ('ALogP (#3)', 'Bond Count'),
       ('ALogP (#3)', 'Bond Polarizabilities'), ('ALogP (#3)', 'Chi0v'),
       ('ALogP (#3)', 'Chi1n'), ('ALogP (#3)', 'Chi1v')], dtype=object)

In [11]:
# ## The versions 1.4.1 and older cause the following error: ImportError: cannot import name 'ABCIndexClass' from 'pandas.core.dtypes.generic' 
# ## (/home/jovyan/anaconda3/envs/chemkube/lib/python3.9/site-packages/pandas/core/dtypes/generic.py)
# ## Pandas v1.3 renamed the ABCIndexClass to ABCIndex. The visions dependency of the pandas-profiling package hasn't caught 
# ## up yet, and so throws an error when it can't find ABCIndexClass. Downgrading pandas to the 1.2.x series will resolve the issue.
# # 1. Edit the file "~/[your_conda_env_path]/lib/site-packages/visions/dtypes/boolean.py"
# # 2. Find the row "from pandas.core.dtypes.generic import ABCIndexClass, ABCSeries" and just replace ABCIndexClass for ABCIndex.
# # 3. Save the boolean.py file and enjoy your report!

# from pandas_profiling import ProfileReport
# profileReport = ProfileReport(X_mini)
# rejected_features = list(profileReport.get_rejected_variables())
# print(rejected_features)

In [12]:
# vif_threshold = 10
# vif = pd.DataFrame()
# vif["VIF Factor"] = [variance_inflation_factor(X_mini.values, i) for i in range(X_mini.shape[1])]
# vif["features"] = X_mini.columns.tolist()
# features_to_remove = vif[vif["VIF Factor"]>vif_threshold]["features"].values.tolist()
# print("There are {} features with a VIF greater than {}.".format(len(features_to_remove),vif_threshold))
# vif[vif["VIF Factor"]>vif_threshold]
# ";  ".join(features_to_remove)

In [13]:
# ! pip install streamlit-pandas-profiling

In [14]:
# import streamlit as st
# from streamlit_pandas_profiling import st_profile_report
# pr = X_mini.profile_report()

## 1. Univariate feature selection

In [15]:
from sklearn.feature_selection import SelectKBest, SelectPercentile, f_classif

# n_best_features = 160
# X_best = SelectKBest(f_classif, k=n_best_features).fit(X_train, y_train)
# mask = X_best.get_support() #list of booleans for selected features
# new_feat = [] 
# for bool, feature in zip(mask, X.columns):
#  if bool:
#      new_feat.append(feature)
# print('The {} best features are:{}'.format(n_best_features, new_feat))

In [16]:
percentile = 5
f_best = SelectPercentile(f_classif, percentile = percentile).fit(X, y)
mask = f_best.get_support() #list of booleans for selected features
n_best_features = [] 
for bool, feature in zip(mask, X.columns):
 if bool:
     n_best_features.append(feature)
print('The {} best features are:{}'.format(len(n_best_features), n_best_features))

The 81 best features are:['ALogP (#1)', 'Aromatic Atoms Count', 'Aromatic Bonds Count', 'FractionCSP3', 'Largest Chain', 'Largest Pi Chain', 'MACCS_11', 'MACCS_2', 'MACCS_22', 'MACCS_23', 'MACCS_32', 'MACCS_34', 'MACCS_4', 'MACCS_42', 'MACCS_5', 'MACCS_66', 'MQN17', 'MQN36', 'NumAromaticCarbocycles', 'NumAromaticRings', 'PubChem_198', 'PubChem_202', 'PubChem_203', 'PubChem_204', 'PubChem_213', 'PubChem_216', 'PubChem_217', 'PubChem_221', 'PubChem_224', 'PubChem_241', 'PubChem_247', 'PubChem_248', 'PubChem_263', 'PubChem_273', 'PubChem_274', 'PubChem_278', 'PubChem_281', 'PubChem_282', 'PubChem_286', 'PubChem_289', 'PubChem_296', 'PubChem_297', 'PubChem_303', 'PubChem_311', 'PubChem_317', 'PubChem_325', 'PubChem_329', 'PubChem_332', 'PubChem_335', 'PubChem_336', 'PubChem_357', 'PubChem_361', 'PubChem_365', 'PubChem_379', 'PubChem_386', 'PubChem_391', 'PubChem_411', 'PubChem_417', 'PubChem_432', 'PubChem_435', 'PubChem_440', 'PubChem_444', 'PubChem_447', 'PubChem_465', 'PubChem_497', 'Pu

/home/jovyan/anaconda3/envs/chemkube/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [ 304  305  306  308  309  310  311  312  313  314  315  316  317  319
  320  321  322  323  324  325  326  327  328  330  331  332  333  334
  335  336  429  456  466  484  491  493  494  497  498  499  500  501
  502  503  504  605  628  676  677  688  691  699  731  737  740  743
  776  787  810  821  843  848  854  865  876  887  899  910  921  932
  943  954  961  975  976 1009 1030 1071 1073 1097 1109 1120 1182 1183
 1184 1185 1186 1188 1189 1190 1191 1192 1193 1194 1197 1199 1200 1203
 1204 1208 1215 1216 1222 1223 1224 1230 1231 1233 1234 1235 1236 1237
 1238 1239 1240 1241 1242 1244 1245 1246 1247 1248 1249 1250 1251 1252
 1262 1263 1264 1266 1267 1268 1269 1270 1271 1272 1273 1274 1275 1277
 1278 1279 1280 1281 1282 1283 1284 1289 1290 1293 1294 1295 1296 1297
 1299 1300 1308 1310 1311 1313 1316 1317 1318 1321 1324 1325 1326 1342
 1348

In [17]:
X_best = X[n_best_features]
X_train, X_test, y_train, y_test = train_test_split(X_best, y, test_size=0.3, random_state=42)

In [18]:
## Random Forest
from sklearn.ensemble import RandomForestClassifier
scoring = {"Accuracy": "accuracy", "F1-score": "f1_weighted"}
kfold=3
param_grid_rf = {
    'n_estimators': [100, 200, 300]
    , 'bootstrap': [True]
    , 'max_features': ["auto"]
    , "criterion": ["gini"]
    , "min_impurity_decrease": [0.0, 0.1]
    , "class_weight" : ["balanced"]
    , "ccp_alpha": [0.0, 0.1]
#     , 'scoring': list(scoring.values())
    }
ahr_rfc = RandomForestClassifier(random_state=42)
CV_rfc = GridSearchCV(estimator=ahr_rfc, param_grid=param_grid_rf, cv= kfold)
CV_rfc.fit(X_train, y_train)
CV_rfc.best_params_

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': 'balanced',
 'criterion': 'gini',
 'max_features': 'auto',
 'min_impurity_decrease': 0.0,
 'n_estimators': 200}

In [19]:
CV_rfc.best_score_

0.7957479119210328

In [20]:
len(CV_rfc.best_estimator_.feature_names_in_)

81

## 2. Recursive feature elimination (RFE)

https://machinelearningmastery.com/rfe-feature-selection-in-python/

In [ ]:
from sklearn.feature_selection import RFE

n_features = 81
estimator = RandomForestClassifier(random_state = 42)
selector = RFE(estimator, n_features_to_select=n_features, step=1)
selector = selector.fit(X, y)
rfe_mask = selector.get_support() #list of booleans for selected features
rfe_features = [] 
for bool, feature in zip(rfe_mask, X.columns):
    if bool:
        rfe_features.append(feature)
rfe_features

In [ ]:
from matplotlib import pyplot
print('Optimal number of features :', selector.n_features_)
print('Best features :', rfe_features)
n_features = X.shape[1]
pyplot.figure(figsize=(16,50))
pyplot.barh(range(n_features), estimator.feature_importances_, align='center') 
pyplot.yticks(np.arange(n_features), X.columns.values) 
pyplot.xlabel('Feature importance')
pyplot.ylabel('Feature')
pyplot.show()

## 3. Recursive feature elimination with cross-validation (RFECV)

In [ ]:
from sklearn.feature_selection import RFECV
cv_estimator = RandomForestClassifier(random_state =42)

cv_estimator.fit(X_train, y_train)
cv_selector = RFECV(cv_estimator,cv= 5, step=1,scoring='accuracy')
cv_selector = cv_selector.fit(X_train, y_train)
rfecv_mask  = cv_selector.get_support() #list of booleans
rfecv_features = [] 
for bool, feature in zip(rfecv_mask, X.columns):
    if bool:
         rfecv_features.append(feature)

In [ ]:
from matplotlib import pyplot
print('Optimal number of features :', cv_selector.n_features_)
print('Best features :', rfecv_features)
n_features = X_train.shape[1]
pyplot.figure(figsize=(16,50))
pyplot.barh(range(n_features), cv_estimator.feature_importances_, align='center') 
pyplot.yticks(np.arange(n_features), X_train.columns.values) 
pyplot.xlabel('Feature importance')
pyplot.ylabel('Feature')
pyplot.show()

In [ ]:
# https://machinelearningmastery.com/rfe-feature-selection-in-python/
    
# Really good tutorial. Select the N most features where N is pre-defined or must be estimated